### Prediction Anaylsis

the folling are two models on price and sales prediction using Price_table.csv and Sales_tables.csv. The Model includes data cleansing and prediction.

### Price Prediction Model

In [25]:
import pandas as pd
import numpy as np


# Load the data
data = pd.read_csv('Price_table.csv')

# Group data by Genmodel_ID
grouped_data = data.groupby('Genmodel_ID')

# Data Cleansing
# Ensure each group has price data up to 2021, considering 4% inflation each year
for name, group in grouped_data:
    last_year = group['Year'].max()
    last_price = group.loc[group['Year'] == last_year, 'Entry_price'].values[0]
    
    for year in range(last_year + 1, 2022):
        last_price *= 1.04  # Apply 4% inflation
        new_row = {'Maker': group['Maker'].iloc[0], 'Genmodel': group['Genmodel'].iloc[0], 'Genmodel_ID': name, 'Year': year, 'Entry_price': last_price}
        data = pd.concat([data, pd.DataFrame([new_row])], ignore_index=True)

# Group data again after adding missing years
grouped_data = data.groupby('Genmodel_ID')

# Initialize an empty list to collect all DataFrames
all_groups = []

# Iterate through each group and calculate the future EMA
for name, group in grouped_data:
    group = group.sort_values('Year')
    last_year = group['Year'].max()
    last_price = group.loc[group['Year'] == last_year, 'Entry_price'].values[0]
    last_ema = group['Entry_price'].ewm(alpha=alpha, adjust=False).mean().iloc[-1]
    
    # Initialize a DataFrame to hold future projections
    future_projections = pd.DataFrame()
    
    for i in range(1, 6):  # Project for the next 5 years
        new_price = last_price * 1.04  # Apply 4% inflation to the last price
        new_ema = alpha * new_price + (1 - alpha) * last_ema  # Calculate the new EMA
        
        # Create a new row with the predicted values
        new_row = pd.DataFrame({
            'Maker': [group['Maker'].iloc[0]],
            'Genmodel': [group['Genmodel'].iloc[0]],
            'Genmodel_ID': [name],
            'Year': [last_year + i],
            'Entry_price': [new_price],
            'EMA': [new_ema]
        })
        
        # Append the new row to the future projections DataFrame
        future_projections = pd.concat([future_projections, new_row], ignore_index=True)
        
        # Update last_price and last_ema for the next iteration
        last_price = new_price
        last_ema = new_ema

    # Concatenate the original group with the future projections
    group = pd.concat([group, future_projections], ignore_index=True)
    
    # Append the updated group to the list
    all_groups.append(group)

# Combine all updated groups back into a single DataFrame
projected_data = pd.concat(all_groups)

# Pivot the DataFrame to get each Genmodel_ID in a single row with years as columns
pivot_data = projected_data.pivot_table(index=['Maker', 'Genmodel', 'Genmodel_ID'], columns='Year', values='EMA', aggfunc='first')

# Reset the index to turn index columns back into regular columns
pivot_data.reset_index(inplace=True)

# Save the updated data to a new CSV file
pivot_data.to_csv('Projected_Price_data.csv', index=False)




### Sales Prediction Model

In [35]:
import pandas as pd
import numpy as np

# Load the sales data
data = pd.read_csv('Sales_table.csv')

# Define alpha, the number of projected years, and noise level
alpha = 0.3
projected_years = 6
noise_level = 0.05  # Adjust this value to control the amount of randomness

# Ensure the columns are sorted correctly by year
# Assuming the years are from 2020 in column 4 (3rd index) to 2001 in the last column
year_columns = data.columns[3:]  # Extract the year columns
year_columns_sorted = sorted(year_columns, key=lambda x: int(x), reverse=True)

# Reorder the columns, keeping the first three columns intact
data = data[data.columns[:3].tolist() + year_columns_sorted]

# Initialize an empty list to collect all DataFrames
all_groups = []

# Iterate through each row and calculate the future EMA
for index, row in data.iterrows():
    sales_data = row[year_columns_sorted].values.astype(float)
    
    # Reverse the sales data array
    reversed_sales_data = np.flip(sales_data)
    
    ema = pd.Series(reversed_sales_data).ewm(alpha=alpha, adjust=False).mean().values
    last_ema = ema[-1]  # Get the last EMA value
    second_last_ema = ema[-2]  # Get the second last EMA value
    
    # Initialize a list to hold future projections
    future_projections = []
    
    # Calculate the EMA for the first projected year (2021)
    new_ema = alpha * second_last_ema + (1 - alpha) * last_ema
    new_ema += np.random.randn() * noise_level * new_ema  # Add random noise
    future_projections.append(new_ema)
    
    last_ema = new_ema
    
    # Project for the next 5 years (2022-2026)
    for i in range(2, 7):
        new_sales = new_ema
        new_ema = alpha * new_sales + (1 - alpha) * last_ema  # Calculate the new EMA
        new_ema += np.random.randn() * noise_level * new_ema  # Add random noise
        future_projections.append(new_ema)
        last_ema = new_ema
    
    # Append the future projections to the row
    row_projection = pd.Series(future_projections, index=[str(year) for year in range(2021, 2021 + projected_years)])
    updated_row = pd.concat([row, row_projection])
    
    # Append the updated row to the list
    all_groups.append(updated_row)

# Combine all updated rows back into a single DataFrame
projected_data = pd.DataFrame(all_groups)

# Save the updated data to a new CSV file
projected_data.to_csv('Projected_Sales_data.csv', index=False)

# Display the combined data
print(projected_data)


      Maker      Genmodel Genmodel_ID   2020   2019   2018   2017   2016  \
0    ABARTH    ABARTH 124         2_1      0     19     27     60      0   
1    ABARTH    ABARTH 500         2_2      0      0      1      2     66   
2    ABARTH    ABARTH 595         2_4   2144   2866   3907   3295   3132   
3    ABARTH    ABARTH 695         2_6     45     65    270    114     29   
4    ABARTH  ABARTH PUNTO         2_9      0      0      0      0      0   
..      ...           ...         ...    ...    ...    ...    ...    ...   
768   VOLVO    VOLVO XC40       96_20  24281  14894   6616      8      0   
769   VOLVO    VOLVO XC60       96_16   7694  11182  10840  14994  14808   
770   VOLVO    VOLVO XC70       96_17      0      0      0     17   1006   
771   VOLVO    VOLVO XC90       96_18   4969   7495   6475   5564   5254   
772   ZENOS     ZENOS E10        99_1      0      0      0      2     10   

      2015  2014  ...  2004  2003  2002  2001          2021          2022  \
0        0